<a href="https://colab.research.google.com/github/sevincsemih76/01-02-exploring-the-relationship-between-gender-and-policing/blob/master/The_last_BBC_news_Data_set_ipynb_adl%C4%B1_not_defterinin_kopyas%C4%B1_adl%C4%B1_not_defterinin_kopyas%C4%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %%capture
# Gerekli kütüphaneleri yükle
!pip install -U transformers accelerate bitsandbytes pandas kagglehub jsonlines

import os
import json
import pandas as pd
import numpy as np
import glob
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import kagglehub
from google.colab import userdata







     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 143.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 149.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.4/160.4 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: kagglehub
    Found existing installation: kagglehub 0.3.13
    Uninstalling kagglehub-0.3.13:
      Successfully uninstalled kagglehub-0.3.13
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.3
    Uninstalling transformers-4.5

In [ ]:
# ============================================================================
# 1. MODELİ YÜKLE (TAMAMEN ÜCRETSİZ - GPU GEREKTİRİR)
# ============================================================================

print("Ücretsiz model yükleniyor (Bu işlem birkaç dakika sürebilir)...")

model_id = "microsoft/Phi-3-mini-4k-instruct" # Hafif ve hızlı bir model
# Eğer vram yetmezse veya daha iyi sonuç istersen "meta-llama/Llama-3.1-8B-Instruct" deneyebilirsin

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)


Ücretsiz model yükleniyor (Bu işlem birkaç dakika sürebilir)...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
# ============================================================================
# 2. VERİ SETİNİ İNDİRME VE YÜKLEME
# ============================================================================

print("\nBBC News veri seti indiriliyor...")
dataset_path = kagglehub.dataset_download("pariza/bbc-news-summary")

def load_data(base_path):
    print(f"Downloaded dataset base path: {base_path}")

    articles_dir = os.path.join(base_path, "bbc-news-summary", "BBC News Summary", "News Articles")
    summaries_dir = os.path.join(base_path, "bbc-news-summary", "BBC News Summary", "Summaries")

    if not os.path.exists(articles_dir) or not os.path.exists(summaries_dir):
        # Fallback if the expected nested structure is not found
        # Try to find a common parent for 'News Articles' and 'Summaries'
        for root, dirs, files in os.walk(base_path):
            if "News Articles" in dirs and "Summaries" in dirs:
                articles_dir = os.path.join(root, "News Articles")
                summaries_dir = os.path.join(root, "Summaries")
                break
        if not os.path.exists(articles_dir) or not os.path.exists(summaries_dir):
            raise FileNotFoundError(f"'News Articles' veya 'Summaries' klasörleri bulunamadı: {base_path}")

    data = []
    for category_folder in os.listdir(articles_dir):
        category_articles_path = os.path.join(articles_dir, category_folder)
        category_summaries_path = os.path.join(summaries_dir, category_folder)

        if os.path.isdir(category_articles_path) and os.path.isdir(category_summaries_path):
            for article_file in os.listdir(category_articles_path):
                if article_file.endswith('.txt'):
                    article_id = os.path.splitext(article_file)[0]
                    story_path = os.path.join(category_articles_path, article_file)
                    summary_path = os.path.join(category_summaries_path, article_file)

                    if os.path.exists(summary_path):
                        try:
                            with open(story_path, 'r', encoding='utf-8', errors='ignore') as f:
                                story_content = f.read()
                            with open(summary_path, 'r', encoding='utf-8', errors='ignore') as f:
                                summary_content = f.read()
                            data.append({'category': category_folder, 'article_id': article_id, 'story': story_content, 'summary': summary_content})
                        except Exception as e:
                            print(f"Dosya okuma hatası {article_file} ({category_folder}): {e}")
                    else:
                        print(f"Uyarı: Özet dosyası bulunamadı: {summary_path}")

    if not data:
        raise ValueError("Hiç veri yüklenemedi. Dosya yollarını ve içeriklerini kontrol edin.")

    return pd.DataFrame(data)

df = load_data(dataset_path)

# Sütun isimlerini standartlaştır (zaten bu isimlerle yüklendiği için kontrol gerekmez)
# Sadece emin olmak için kontrol edilebilir
if 'summary' not in df.columns:
    print("Uyarı: 'summary' sütunu bulunamadı.")
if 'story' not in df.columns:
    print("Uyarı: 'story' sütunu bulunamadı.")

print(f"Veri seti {len(df)} satır ile başarıyla yüklendi.")
print("İlk 5 satır:")
print(df.head())



BBC News veri seti indiriliyor...
Using Colab cache for faster access to the 'bbc-news-summary' dataset.
Downloaded dataset base path: /kaggle/input/bbc-news-summary
Veri seti 2225 satır ile başarıyla yüklendi.
İlk 5 satır:
   category article_id                                              story  \
0  politics        361  Budget to set scene for election\n\nGordon Bro...   
1  politics        245  Army chiefs in regiments decision\n\nMilitary ...   
2  politics        141  Howard denies split over ID cards\n\nMichael H...   
3  politics        372  Observers to monitor UK election\n\nMinisters ...   
4  politics        333  Kilroy names election seat target\n\nEx-chat s...   

                                             summary  
0  - Increase in the stamp duty threshold from £6...  
1  "They are very much not for the good and will ...  
2  Michael Howard has denied his shadow cabinet w...  
3  The report said individual registration should...  
4  UKIP's leader, Roger Knapman, has 

In [ ]:
# ============================================================================
# 3. SORU-CEVAP OLUŞTURMA FONKSİYONU
# ============================================================================

def generate_free_qa(text):
    prompt = f"""<|user|>
Extract 3-6 factual question-answer pairs from the text. Short answers only. Format as JSON with 'questions' and 'answers' lists. Text: {text[:1500]}<|end|>
<|assistant|>"""

    # Hata düzeltmesi: 'DynamicCache' object has no attribute 'seen_tokens' hatasını önlemek için use_cache=False eklendi.
    output = pipe(prompt, max_new_tokens=500, do_sample=True, temperature=0.1, use_cache=False)
    response = output[0]['generated_text'].split("<|assistant|>")[-1].strip()

    try:
        # JSON temizliği
        start = response.find('{')
        end = response.rfind('}') + 1
        result = json.loads(response[start:end])
        return result.get("questions", []), result.get("answers", [])
    except:
        return [], []

In [ ]:
# ============================================================================
# 4. İŞLEME VE KAYDETME (TEST: İLK 10 SATIR)
# ============================================================================

test_df = df.copy()
qs_list, ans_list = [], []

print(f"\n{len(test_df)} satır ücretsiz modelle işleniyor...")

for idx, row in test_df.iterrows():
    text = str(row['summary'])
    q, a = generate_free_qa(text)
    qs_list.append(q)
    ans_list.append(a)
    print(f"İlerleme: {idx+1}/{len(test_df)}")

test_df['questions'] = qs_list
test_df['answers'] = ans_list

test_df.to_csv("BBCnews_QA_Free.csv", index=False)
print("\nİşlem tamamlandı! 'BBCnews_QA_Free.csv' dosyası oluşturuldu.")


2225 satır ücretsiz modelle işleniyor...


İlerleme: 1/2225
İlerleme: 2/2225
İlerleme: 3/2225
İlerleme: 4/2225
İlerleme: 5/2225
İlerleme: 6/2225
İlerleme: 7/2225
İlerleme: 8/2225
İlerleme: 9/2225


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


İlerleme: 10/2225
İlerleme: 11/2225
İlerleme: 12/2225
İlerleme: 13/2225
İlerleme: 14/2225
İlerleme: 15/2225
İlerleme: 16/2225
İlerleme: 17/2225
İlerleme: 18/2225
İlerleme: 19/2225
İlerleme: 20/2225
İlerleme: 21/2225
İlerleme: 22/2225
İlerleme: 23/2225
İlerleme: 24/2225
İlerleme: 25/2225
İlerleme: 26/2225
İlerleme: 27/2225
İlerleme: 28/2225
İlerleme: 29/2225
İlerleme: 30/2225
İlerleme: 31/2225
İlerleme: 32/2225
İlerleme: 33/2225
İlerleme: 34/2225
İlerleme: 35/2225
İlerleme: 36/2225
İlerleme: 37/2225
İlerleme: 38/2225
İlerleme: 39/2225
İlerleme: 40/2225
İlerleme: 41/2225
İlerleme: 42/2225
İlerleme: 43/2225
İlerleme: 44/2225
İlerleme: 45/2225
İlerleme: 46/2225
İlerleme: 47/2225
İlerleme: 48/2225
İlerleme: 49/2225
İlerleme: 50/2225
İlerleme: 51/2225
İlerleme: 52/2225
İlerleme: 53/2225
İlerleme: 54/2225
İlerleme: 55/2225
İlerleme: 56/2225
İlerleme: 57/2225
İlerleme: 58/2225
İlerleme: 59/2225
İlerleme: 60/2225
İlerleme: 61/2225
İlerleme: 62/2225
İlerleme: 63/2225
İlerleme: 64/2225
İlerleme: 